<a href="https://colab.research.google.com/github/hamsungmin/DataTrainAnalysis/blob/main/project_week13_PDF%EC%9E%90%EB%A3%8C_%EB%B6%84%EC%84%9D_OCR_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. PDF 문서의 레이아웃 분석 및 OCR 처리 설계**

**진행 프로세스**

제조업체의 **품질관리 문서(PDF)** 는 문서의 생성 방식에 따라 텍스트 중심, 이미지 중심, 표 중심 등 다양한 형태로 존재하며,  **품질분석 GPT 챗봇**을 개발하는 과정에서, 이 PDF 자료들을 자동으로 구조화해야 합니다.

- PDF 자료를 정확히 추출·분석하기 위해 활용할 수 있는 **파이썬 모듈**을  간단히 정리합니다.
    - pdfplumber /  img2table[easyocr]  / PyMuPDF (fitz) / img2table / pdf2image
- 품질검수보고서.pdf’ 문서의 레이아웃을 **딕셔너리 형태로 추출**하는 프로그램을 작성합니다.
    - 각 페이지별 **텍스트·이미지·표 정보를 수치로 요약**
    - 페이지별 구조를 기반으로 **OCR 적용이 필요한 페이지를 구분**할 수 있도록 설계

In [ ]:
import pdfplumber
import fitz  # PyMuPDF
import json
from img2table.document import PDF
from img2table.ocr import TesseractOCR # 또는 EasyOCR
import pandas as pd # img2table 결과 처리를 위해 필요
import pytesseract # pytesseract 불러오기 추가
import io # io 모듈 임포트 추가
from PIL import Image # PIL.Image 임포트 추가
import numpy as np # numpy 임포트 추가
import tempfile # 임시 파일 사용을 위해 임포트

def analyze_pdf_layout(pdf_path):

    layout_summary = {}
    pdf_document_img2table = None # img2table 문서 객체 초기화
    ocr = None # ocr 객체 초기화

    try:
        # img2table을 위한 OCR 인스턴스 생성 (Tesseract 사용)
        # EasyOCR 사용시 ocr = EasyOCR(lang=['ko', 'en'])
        # Tesseract 실행 파일 경로 설정 (Colab 환경에 맞게 설정)
        try:
            pytesseract.tesseract_cmd = r'/usr/bin/tesseract' # Colab 기본 경로
            ocr = TesseractOCR(lang="kor+eng")
            # img2table 1.4.2 버전에서는 PDF 객체를 바로 사용하여 페이지를 지정하는 방식이 변경되었을 수 있습니다.
            # PDF 객체 생성은 그대로 두고, 이미지 추출 로직을 사용합니다.
            # pdf_document_img2table = PDF(pdf_path) # 이 객체는 image_to_tables 사용 시 필요 없습니다.
        except Exception as e:
             print(f"img2table 또는 Tesseract 초기화 중 오류 발생: {e}")
             print("이미지 표 감지 기능이 작동하지 않을 수 있습니다.")
             ocr = None


        with pdfplumber.open(pdf_path) as pdf:
            pdf_document = fitz.open(pdf_path)
            total_pages = len(pdf.pages)


            for i in range(total_pages):
                page = pdf.pages[i]
                fitz_page = pdf_document.load_page(i)

                # 텍스트 추출 (텍스트 문자 수)
                text = page.extract_text() or ""
                text_chars = len(text)

                # pdfplumber로 테이블 감지 (텍스트표로 간주)
                tables = page.find_tables()
                tables_count = len(tables)

                # PyMuPDF로 이미지 감지
                images = fitz_page.get_images(full=True)
                images_count = len(images)

                # img2table을 사용하여 이미지 내 표 감지 시도
                img_tables_count = 0
                if ocr: # OCR 객체가 성공적으로 생성된 경우에만 시도
                    try:
                        pix = fitz_page.get_pixmap()
                        img_bytes = pix.tobytes("png")
                        pil_image = Image.open(io.BytesIO(img_bytes))


                        from img2table.document import Image as Img2TableImage

                        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_img_file:
                            pil_image.save(tmp_img_file.name)
                            temp_file_path = tmp_img_file.name

                        # Initialize Img2TableImage with the file path
                        img2table_image = Img2TableImage(temp_file_path)
                        detected_img_tables = img2table_image.extract_tables(ocr=ocr) # 이미지에서 테이블 추출

                        img_tables_count = len(detected_img_tables)

                        import os
                        os.remove(temp_file_path)


                    except Exception as img_table_e:
                         # 이미지 표 감지 중 오류가 발생해도 계속 진행
                         print(f"{i+1}페이지 이미지 표 감지 (img2table) 중 오류 발생: {img_table_e}")
                         img_tables_count = 0 # 오류 발생 시 0으로 설정


                page_info = {
                    "텍스트": text_chars,
                    "이미지": images_count,
                    "텍스트표": tables_count,
                    "이미지표": img_tables_count # img2table 감지 결과 반영
                }

                layout_summary[f"{i+1}페이지"] = page_info

            pdf_document.close()
            return layout_summary # 결과를 반환하도록 수정

    except FileNotFoundError:
        print(f"Error: File not found at {pdf_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# 분석할 PDF 파일 경로
pdf_file_path = '/content/품질검수보고서.pdf'

# 레이아웃 분석 실행
pdf_layout = analyze_pdf_layout(pdf_file_path)

# 결과 출력
if pdf_layout:
  pdf_layout_json = "pdf_layout.json"
  # --- 결과 저장 ---
  with open(pdf_layout_json, 'w', encoding='utf-8') as f:
      json.dump(pdf_layout, f, indent=4, ensure_ascii=False)
      print(json.dumps(pdf_layout, indent=4, ensure_ascii=False))

{
    "1페이지": {
        "텍스트": 0,
        "이미지": 1,
        "텍스트표": 0,
        "이미지표": 0
    },
    "2페이지": {
        "텍스트": 0,
        "이미지": 1,
        "텍스트표": 0,
        "이미지표": 0
    },
    "3페이지": {
        "텍스트": 0,
        "이미지": 1,
        "텍스트표": 0,
        "이미지표": 0
    },
    "4페이지": {
        "텍스트": 0,
        "이미지": 1,
        "텍스트표": 0,
        "이미지표": 1
    },
    "5페이지": {
        "텍스트": 0,
        "이미지": 1,
        "텍스트표": 0,
        "이미지표": 1
    }
}


In [ ]:
!pip install img2table easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 28.4 MB/s eta 0:00:00


**2. 텍스트 추출**

**진행 프로세스**

        
- ‘품질검수보고서.pdf’ 의 구조를 파악한 layout(1번문제결과)를 이용하여  각 페이지의 이미지를 인식하고 글자와 표를 구분하여 텍스트 정보를 추출합니다.
이렇게 수집된 텍스트와 표 정보를 구조화하여, ocr_report.json 파일로 저장함으로써, 이후 분석이나 보고서 작성에 활용할 수 있도록 준비합니다.
        
    - 문서 전체에서 읽어들인 텍스트를 페이지별로 정리
    - 표가 포함된 페이지는 별도로 감지하여 표의 내용까지 함께 저장

In [ ]:
import json
import pdfplumber
import pytesseract
from PIL import Image
import fitz  # PyMuPDF
import io
import tempfile
import os
from img2table.document import Image as Img2TableImage
from img2table.ocr import TesseractOCR


def extract_pdf_ocr(pdf_path, output_json='ocr_report.json'):
    pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
    ocr_engine = TesseractOCR(lang="kor+eng")

    pdf_document = fitz.open(pdf_path)
    pdf_plumber = pdfplumber.open(pdf_path)

    ocr_report = {
        "파일명": os.path.basename(pdf_path),
        "레이아웃요약": {},
        "텍스트추출결과": {},
        "표추출결과": {}
    }

    for i in range(len(pdf_document)):
        page_num = i + 1
        page_fitz = pdf_document.load_page(i)
        page_plumber = pdf_plumber.pages[i]

        # --- 기본값 ---
        page_summary = {"텍스트": 0, "이미지": 0, "텍스트표": 0, "이미지표": 0}
        text_content = ""
        table_results = []

        # --- 1. 일반 텍스트 추출 ---
        plumber_text = page_plumber.extract_text()
        if plumber_text:
            text_content = plumber_text.strip()
            page_summary["텍스트"] = 1

        # --- 2. 페이지에 이미지 존재 확인 ---
        images = page_fitz.get_images(full=True)
        if images:
            page_summary["이미지"] = len(images)

        # --- 3. 텍스트 기반 표 추출 ---
        plumber_tables = page_plumber.find_tables()
        if plumber_tables:
            page_summary["텍스트표"] = len(plumber_tables)
            for table in plumber_tables:
                data = table.extract()
                table_results.append(data)

        # --- 4. 이미지 내 표 추출 (img2table 사용) ---
        try:
            # 고해상도 렌더링
            pix = page_fitz.get_pixmap(matrix=fitz.Matrix(2, 2))
            img_bytes = pix.tobytes("png")
            pil_image = Image.open(io.BytesIO(img_bytes))

            with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_img:
                pil_image.save(tmp_img.name)
                temp_path = tmp_img.name

            img_doc = Img2TableImage(temp_path)
            img_tables = img_doc.extract_tables(ocr=ocr_engine)

            if img_tables:
                page_summary["이미지표"] = len(img_tables)
                for img_table in img_tables:
                    df_dict = img_table.df.to_dict('records')
                    table_results.append(df_dict)

            os.remove(temp_path)

        except Exception as e:
            print(f"{page_num}페이지 이미지 표 추출 중 오류: {e}")

        # --- 5. OCR 텍스트 추출 (이미지에만 적용) ---
        # 이미지 내 텍스트만 OCR (표는 위에서 별도로 처리)
        try:
            ocr_text = pytesseract.image_to_string(pil_image, lang='kor+eng', config='--psm 6').strip()
            if ocr_text and not text_content:
                text_content = ocr_text
        except Exception as e:
            print(f"{page_num}페이지 OCR 오류: {e}")

        # --- 결과 저장 ---
        ocr_report["레이아웃요약"][str(page_num)] = page_summary
        if text_content:
            ocr_report["텍스트추출결과"][str(page_num)] = text_content
        if table_results:
            ocr_report["표추출결과"][str(page_num)] = table_results

    # --- 결과 저장 ---
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(ocr_report, f, indent=4, ensure_ascii=False)

    pdf_document.close()
    pdf_plumber.close()

    print(f"✅ 결과가 '{output_json}'로 저장되었습니다.")
    return ocr_report


# 실행 예시
pdf_path = "/content/품질검수보고서.pdf"
ocr_result = extract_pdf_ocr(pdf_path)


✅ 결과가 'ocr_report.json'로 저장되었습니다.


**3.  판정별 품질검수 요약 보고서 작성 _ 1,2번 미진행시프로젝트폴더의 결과 json으로 하세요**

<aside>

**진행 프로세스**

- 품질검수 문서의 OCR 결과(`ocr_report.json`)에는 각 페이지별로 검사 내용과 판정 결과가 포함되어 있습니다. 이 중 ‘최종 판정’ 문장은 문서마다 반복적으로 나타나며, 이를 먼저 추출해야 전체 문서에서 어떤 판정 유형이 존재하는지 파악할 수 있습니다. **고유한 판정값**(예: 합격, 폐기, 불량 등)을 식별하기 위하여 중복된 판정 문장을 제거합니다.
    - ‘ocr_report.json’에서 각 페이지에 포함된 **‘최종 판정’ 문장** 출력
    - 중복된 판정 문장을 제거하고, 고유한 판정 결과확인
- ocr_report.json의 각 페이지에서 시리얼 번호와 최종 판정 정보를 이용하여, 같은 판정을 받은 항목끼리 묶어 정리된 **품질검수 보고서**를 작성합니다.
    - 각 그룹에 포함된 페이지 번호와 시리얼 번호를 정리
    - 판정 유형별로 건수 요약과 세부 내역을 표 형식으로 출력
</aside>

In [ ]:
import json
import re
from collections import defaultdict

def summarize_quality_report_console(ocr_json_path):
    with open(ocr_json_path, 'r', encoding='utf-8') as f:
        ocr_data = json.load(f)

    text_results = ocr_data.get("텍스트추출결과", {})
    summary_data = []

    for page_num, text in text_results.items():
        # 시리얼 번호 추출
        serial_match = re.search(r"시리얼\s*번호[:：]?\s*([A-Za-z0-9\-]+)", text)
        serial_no = serial_match.group(1) if serial_match else "N/A"

        # OCR 오류를 포함한 '최종 판정' 탐지
        verdict_match = re.search(r"[최췌죄촤좨쵀쵸췽]\s*종?\s*판정[:：]?\s*([^\n]+)", text)
        verdict_text = verdict_match.group(1).strip() if verdict_match else "N/A"

        summary_data.append({
            "페이지": page_num,
            "시리얼번호": serial_no,
            "최종판정": verdict_text
        })

    # 고유 판정 목록
    unique_verdicts = sorted(list({item["최종판정"] for item in summary_data if item["최종판정"] != "N/A"}))

    print("📘 [고유한 판정 결과 목록]")
    for v in unique_verdicts:
        print(f" - {v}")

    # 판정별 그룹화
    grouped_data = defaultdict(list)
    for item in summary_data:
        grouped_data[item["최종판정"]].append(item)

    # 출력
    print("\n📊 [판정별 품질검수 요약 보고서]")
    for verdict in unique_verdicts:
        items = grouped_data[verdict]
        print(f"\n▶ 판정: {verdict} (총 {len(items)}건)")
        print("--------------------------------------------------")
        for it in items:
            print(f"페이지 {it['페이지']:>2} | 시리얼번호: {it['시리얼번호']}")

    # 통계
    print("\n📈 [요약 통계]")
    for verdict in unique_verdicts:
        print(f" - {verdict}: {len(grouped_data[verdict])}건")

# 위에서 진행한 orc_report는 이미지 -> 텍스트 정확도가 떨어져 정답지를 이용해서 판단
# 실행 예시
json_path = "/content/ocr_report_정답지.json"
summarize_quality_report_console(json_path)


📘 [고유한 판정 결과 목록]
 - 불량 (반품 처리)
 - 폐기
 - 합격

📊 [판정별 품질검수 요약 보고서]

▶ 판정: 불량 (반품 처리) (총 1건)
--------------------------------------------------
페이지  5 | 시리얼번호: B20251017-002

▶ 판정: 폐기 (총 1건)
--------------------------------------------------
페이지  3 | 시리얼번호: A20251017-001

▶ 판정: 합격 (총 2건)
--------------------------------------------------
페이지  2 | 시리얼번호: B20251017-001
페이지  4 | 시리얼번호: A20251017-002

📈 [요약 통계]
 - 불량 (반품 처리): 1건
 - 폐기: 1건
 - 합격: 2건


4. **GPT_API을 활용한 분석 보고서 자동화
_ 1,2번 미진행시프로젝트폴더의 결과 json으로 하세요**

<aside>

**진행 프로세스**

- GPT API를 이용하여  **`ocr_report.json`**(문서 내용)과 **`pdf_layout.json`**(문서 구조)를 이용하여 품질검수 보고서를 자동으로 해석하고, 그 결과를 바탕으로 **GPT가 스스로 품질 리포트를 작성**하도록 합니다.
이 과정에서  AI는 문서의 구조를 파악하고, 검사 결과와 불량 원인을 찾아내며, 표에 포함된 수치를 해석하여 품질 수준과 개선점을 도출할수 있습니다.
    - ’ocr_report.json’ , ‘pd_layout.json’ 을 종합해 문서 개요 → 검사 결과 요약 → 불량 원인 분석 → 성능지표 해석 → 개선 제안 생성

**결과물**

</aside>

**📊 검사 결과 요약**
| 구분       | 시리얼 번호        | Lot 번호       | 담당자          | 검사 항목 수          | 주요 결과         | 최종 판정     |
| -------- | ------------- | ------------ | ------------ | ---------------- | ------------- | --------- |
| 정상 출고 승인 | B20251017-001 | T25-BETA-03  | 이서아 (주임 연구원) | 외관, 기능, 성능 종합    | 전 항목 기준치 충족   | ✅ 합격      |
| 부적합품     | A20251017-001 | T25-ALPHA-01 | 김민준 (선임 연구원) | 기능 중 성능 실패       | 제어 회로 과열 소손   | ❌ 폐기      |
| 정상 제품    | A20251017-002 | T25-BETA-03  | 박도운 (사원)     | 6개 항목            | 모든 측정값 정상     | ✅ 합격      |
| 불량 제품    | B20251017-002 | T25-BETA-03  | 최지우 (사원)     | 6개 항목 중 기능·성능 실패 | 외관 파손으로 검사 중단 | ❌ 불량 (반품) |

*외관 검사 결과:*

* 대부분 양호하나, B20251017-002 제품에서 상단 3cm 균열 발생

*기능 검사 결과:*

* 정상 제품은 모두 기준치 내 반응(0.7~0.8초), 타이머 오차 +0.59sec 이내.

* 불량 제품은 외관 손상으로 측정 중단.

*성능 검사 결과:*

* 정상 제품: 온도 및 소음 기준 충족 (820℃, 35dB)

* 부적합품: PCB 커패시터 과열 소손 → 안전 이슈 발생.


**🧪 불량 원인 분석**

*A20251017-001 (폐기)*

* 불량 현상: 예열 중 커패시터(C-102) 과열로 소손 발생

* 원인 추정: 공급 부품 내구성 결함 가능성

* 영향 범위: 동일 Lot 전량 리콜 요청, 공급사 클레임 접수

*B20251017-002 (불량)*

* 불량 현상: 외관 균열로 기능·성능 검사 불가

* 원인 추정: 포장 또는 운송 중 충격 발생 가능성

* 영향 범위: 출하 프로세스 개선 필요

**📈 성능 지표 해석**
| 항목       | 기준     | 측정값 (정상 제품) | 결과 | 해석                |
| -------- | ------ | ----------- | -- | ----------------- |
| 예열 시간    | ≤ 3분   | 2분 30초      | 정상 | 기준 대비 83% 수준으로 양호 |
| 최고 동작 온도 | ≤ 900℃ | 859℃        | 정상 | 안정적 열 제어          |
| 소비 전력    | ≤ 900W | 850W        | 정상 | 규격 내 동작           |
| 소음       | ≤ 40dB | 35dB        | 정상 | 정숙성 우수            |
| 전원 반응 시간 | ≤ 1초   | 0.7~0.8초    | 정상 | 빠른 반응 속도          |


* 성능 안정성 판단: 기준 대비 여유가 충분하고, 생산 로트 간 성능 편차도 작음

* 불량품 제외 시 전체 합격률: 75% (3/4)

**🛠️ 개선 제안**

1. 부품 품질 관리 강화

*  공급사 커패시터 내구성 시험 항목 강화 및 입고 검증 절차 추가

* Lot 추적 관리 및 불량 발생 시 선제적 리콜 체계 확립

2. 운송 및 포장 공정 개선

* 충격 보호 완충재 보강

* 포장 전후 외관 검사 공정 강화

* 충격 센서(Shock Indicator) 부착 고려

3. 검수 프로세스 자동화 고도화

* OCR 기반 자동 판정 시스템 적용으로 인적 오류 최소화

* 불량 데이터 자동 집계 및 대시보드화

4. 지속적 성능 모니터링 체계

* 예열 시간 및 온도 편차를 주기적으로 추적 → 공정 이상 조기 감지

* 불량 발생 이력 DB화 및 AI 예측 모델 적용 가능

**🏁 결론**

본 품질검수 보고서에 따르면 정상 제품은 전반적으로 성능과 품질이 안정적이며, 불량은 특정 부품의 결함과 운송 과정에서의 외관 손상이 주된 원인입니다.
특히 PCB 커패시터 소손은 안전과 직결되는 중대 결함으로 공급망 품질 강화가 시급합니다.
또한 외관 손상은 출하 과정 개선으로 충분히 예방 가능하므로 물류 및 포장 공정 강화가 필요합니다.
